In [30]:
# Necessary imports
import findspark
findspark.init() # Find Spark installation

from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, DoubleType, StringType, BooleanType, TimestampType, StructType, StructField

# For ML tasks (even if demonstrating MapReduce concepts, preprocessing often uses MLlib)
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler, Bucketizer
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors, VectorUDT, DenseVector
from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix # For potential matrix operations

import math
import heapq
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 

import sys
from pathlib import Path

sys.path.append(str(Path("../..").resolve()))

from src.data_ingestion import *
from src.data_preprocessing import *
from src.descriptive_analytics import *

from pyspark.sql import DataFrame
from pyspark.sql.functions import col
from pyspark.sql import functions as F

import seaborn as sns

import numpy as np

from itertools import combinations

from scipy import stats

import matplotlib.pyplot as plt

import pandas as pd
from pyspark.sql.window import Window

from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.linalg import Vectors




In [31]:
spark = init_spark()
df = load_data(spark, "./sample2000_data.csv")
df.show(5)


+-------+-------+--------+-------------------+-------------------+-----------------+-----------+-------+-------+------------+--------------------+---------------+----------------+------------+-----+----------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+
|     ID| Source|Severity|         Start_Time|           End_Time|        Start_Lat|  Start_Lng|End_Lat|End_Lng|Distance(mi)|         Description|         Street|            City|      County|State|   Zipcode|Country|  Timezone|Airport_Code|  Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Precipitation(in)|Weather_Condition|Amenity| 

In [32]:
import math
from collections import Counter
from pyspark.sql.functions import col
from pyspark.sql.types import ByteType, ShortType, IntegerType, LongType, FloatType, DoubleType

# Filter numeric columns
numeric_df = df.select([ 
    col(c).cast("double").alias(c) 
    for c in df.columns 
    if isinstance(df.schema[c].dataType, (ByteType, ShortType, IntegerType, LongType, FloatType, DoubleType))
])

# Split data into train and test
train_data, test_data = numeric_df.randomSplit([0.8, 0.2], seed=42)

# Convert the data to RDD
train_rdd = train_data.rdd
test_rdd = test_data.rdd

# Broadcast train data
train_broadcast = spark.sparkContext.broadcast(train_rdd.collect())

# Define Euclidean distance function
# Define Euclidean distance function with None handling
def euclidean_distance(row1, row2):
    dist = 0.0
    for i in range(len(row1) - 1):
        if row1[i] is not None and row2[i] is not None:  # Handle None values
            dist += (row1[i] - row2[i]) ** 2
        else:
            # Handle missing values by skipping or assigning a default value (e.g., 0)
            dist += 0  # Or you could choose a different strategy like using the mean or median
    return math.sqrt(dist)
# Set the number of neighbors
k = 3

def mapper(row):
    distances = []
    train_data = train_broadcast.value  # get chunk of data
    for train_row in train_data:
        if len(row) == len(train_row):  # Ensure rows have the same length
            dist = euclidean_distance(row[:-1], train_row[:-1])  # Exclude label for distance calc
            distances.append((dist, train_row[-1]))  # Append class label as the second value
    distances.sort(key=lambda x: x[0])
    return distances[:k]   # idx:[(dist, class)]

# Calculate the distance of each row in test data to all rows in train data
results = test_rdd.map(lambda row: mapper(row))

def reducer(distance):
    classes = [dist[1] for dist in distance]
    most_common_class = Counter(classes).most_common(1)[0][0]
    return most_common_class    # idx:(class)

# Get the most common class for each row in test data
predictions = results.map(lambda row: reducer(row))

# Get the actual classes
actual = test_rdd.map(lambda row: row[-1])  # The class label is assumed to be the last column
print("Sample row:", test_rdd.take(5))  # Check first 5 rows of the RDD

# Debugging step: Check the first few elements of both RDDs
print(predictions.take(5))
print(actual.take(5))

# Now, try zipping them
zipped_rdd = predictions.zip(actual)

# Filter correct predictions
correct = zipped_rdd.filter(lambda x: x[0] == x[1]).count()

# Calculate accuracy
total_samples = test_rdd.count()
accuracy = correct / float(total_samples)

print("Accuracy: ", accuracy)

spark.stop()


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 59.0 failed 1 times, most recent failure: Lost task 0.0 in stage 59.0 (TID 442) (RAISA-C31-0325 executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:789)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(Interru